In [1]:
from odf.opendocument import OpenDocumentText, unicode
from odf.style import Style, TextProperties
from odf.text import H, P, Span, List, ListItem

textdoc = OpenDocumentText()
# Styles
s = textdoc.styles
h1style = Style(name="Heading 1", family="paragraph")
h1style.addElement(TextProperties(attributes={'fontsize':"24pt",'fontweight':"bold" }))
s.addElement(h1style)
# An automatic style
boldstyle = Style(name="Bold", family="text")
boldprop = TextProperties(fontweight="bold")
boldstyle.addElement(boldprop)
textdoc.automaticstyles.addElement(boldstyle)
# Text
h=H(outlinelevel=1, stylename=h1style, text="My first text")
textdoc.text.addElement(h)
p = P(text="Hello world. ")
boldpart = Span(stylename=boldstyle, text="This part is bold. ")
p.addElement(boldpart)
p.addText("This is after bold.")
textdoc.text.addElement(p)

textdoc.save("../data/reports/myfirstdocument.odt")





In [2]:
from management.service_carges.service_charge_statement import ServiceChargeStatement
scs = ServiceChargeStatement()

In [3]:
scs.load('../data/finance')

In [4]:
def compute_scs_saldo(year, scs_type, advance_bc):
    dwelling_balance = scs.get_service_charge_balance(year, scs_type)
    sc_account = dwelling_balance.get_account('SERVICE_CHARGES')
    advance_payment = sc_account.get_booking_entry_by_booking_code(advance_bc)[0].get_amount()

    scs_costs = 0
    scs_bookings = sc_account.get_bookings()
    for be in scs_bookings:
        if not be.get_booking_code() == advance_bc:
            scs_costs += be.get_amount()

    saldo =  advance_payment - scs_costs
    return scs_costs, advance_payment, saldo

In [5]:
scs_costs, advance, saldo = compute_scs_saldo(2020, 'RENTAL', 'SC15')


In [6]:
def list_entries(year, scs_type, advance_bc):
    balance = scs.get_service_charge_balance(year, scs_type)
    sc_account = balance.get_account('SERVICE_CHARGES')

    result = list()
    scs_bookings = sc_account.get_bookings()
    for be in scs_bookings:
        if not be.get_booking_code() == advance_bc:
            result.append([be.get_name(), be.get_amount()])

    return result

year = 2020
scs_list = list_entries(year, "RENTAL", "SC15")


In [16]:
from odf import opendocument, text, teletype
from odf.opendocument import OpenDocumentText
from odf.style import (Style, TextProperties, ParagraphProperties,
                       ListLevelProperties, TabStop, TabStops)
from odf.text import (H, P, List, ListItem, ListStyle, ListLevelStyleNumber,
                      ListLevelStyleBullet)
from odf import teletype
from odf.table import Table, TableColumn, TableRow, TableCell

from datetime import date
aktuellesDatum = date.today().strftime("%d.%m.%Y")

# Creating different style used in the document
st = textdoc.styles


bulletedliststyle = ListStyle(name="BulletList")
level = 1
bulletlistproperty = ListLevelStyleBullet(level=str(level), bulletchar=u"•")
bulletlistproperty.addElement(ListLevelProperties(
    minlabelwidth="%fcm" % level))
bulletedliststyle.addElement(bulletlistproperty)
st.addElement(bulletedliststyle)


# Creating a tabstop at 10cm
tabstops_style = TabStops()
tabstop_style = TabStop(position="10cm")
tabstops_style.addElement(tabstop_style)
tabstoppar = ParagraphProperties()
tabstoppar.addElement(tabstops_style)
tabparagraphstyle = Style(name="Question", family="paragraph")
tabparagraphstyle.addElement(tabstoppar)
st.addElement(tabparagraphstyle)

tablecontents = Style(name="Table Contents", family="paragraph")
tablecontents.addElement(ParagraphProperties(numberlines="false", linenumber="0"))
textdoc.styles.addElement(tablecontents)



doc = opendocument.load('../data/reports/scs_template.odt')
for item in doc.getElementsByType(text.P):
    s = teletype.extractText(item)

    if s.find('[[date]]') != -1:
        s = s.replace('[[date]]', str(aktuellesDatum))
        new_item = text.P()
        new_item.setAttribute('stylename', item.getAttribute('stylename'))
        new_item.addText(s)
        item.parentNode.insertBefore(new_item, item)
        item.parentNode.removeChild(item)

    if s.find('[[year]]') != -1:
        s = s.replace('[[year]]', str(year))
        new_item = text.P()
        new_item.setAttribute('stylename', item.getAttribute('stylename'))
        new_item.addText(s)
        item.parentNode.insertBefore(new_item, item)
        item.parentNode.removeChild(item)

    if s.find('[[cost]]') != -1:
        s = s.replace('[[cost]]', str(scs_costs))
        new_item = text.P()
        new_item.setAttribute('stylename', item.getAttribute('stylename'))
        new_item.addText(s)
        item.parentNode.insertBefore(new_item, item)
        item.parentNode.removeChild(item)

    if s.find('[[advance]]') != -1:
        s = s.replace('[[advance]]', str(advance))
        new_item = text.P()
        new_item.setAttribute('stylename', item.getAttribute('stylename'))
        new_item.addText(s)
        item.parentNode.insertBefore(new_item, item)
        item.parentNode.removeChild(item)

    if s.find('[[saldo]]') != -1:
        s = s.replace('[[saldo]]', str(saldo))
        new_item = text.P()
        new_item.setAttribute('stylename', item.getAttribute('stylename'))
        new_item.addText(s)
        item.parentNode.insertBefore(new_item, item)
        item.parentNode.removeChild(item)

    if s.find('[[scs_list]]') != -1:
        table = Table()
        table.addElement(TableColumn())
        table.addElement(TableColumn())
        # bulletlist = List(stylename=bulletedliststyle)
        for elem in scs_list:
            # list_elem = ListItem()
            # list_para = P(stylename=tabparagraphstyle)
            # list_content = elem[0]+":\t"+str(elem[1])+' €'
            # teletype.addTextToElement(list_para, list_content)
            # list_elem.addElement(list_para)
            # bulletlist.addElement(list_elem)
            tr = TableRow()
            table.addElement(tr)
            tc = TableCell()
            tr.addElement(tc)
            p = P(stylename=tablecontents,text=elem[0])
            tc.addElement(p)
            tc2 = TableCell()
            tr.addElement(tc2)
            p2 = P(stylename=tablecontents,text=str(elem[1])+ ' €')
            tc2.addElement(p2)

        item.parentNode.insertBefore(table, item)
        item.parentNode.removeChild(item)

doc.save('../data/reports/result.odt')